In [100]:
import psycopg2
import pandas as pd
import ast
import numpy as np
import pickle
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from ray import train, tune, data, serve
from sklearn.ensemble import RandomForestClassifier

In [26]:
r = robjects.r
r['source']('D://Cropnuts/DSML155/splits.R')
split = robjects.globalenv['split']

R[write to console]: prospectr version 0.2.7 -- cakes

R[write to console]: check the package repository at: https://github.com/l-ramirez-lopez/prospectr

R[write to console]: data.table 1.15.4 using 6 threads (see ?getDTthreads).  
R[write to console]: Latest news: r-datatable.com

R[write to console]: Use suppressPackageStartupMessages() to eliminate package startup
messages

R[write to console]: terra 1.7.78

R[write to console]: 
Attaching package: 'terra'


R[write to console]: The following object is masked from 'package:data.table':

    shift


R[write to console]: The following object is masked from 'package:prospectr':

    resample




In [29]:
def get_db_cursor():
    username = "doadmin"
    password = 'yzmodwh2oh16iks6'
    host = 'db-postgresql-cl1-do-user-2276924-0.db.ondigitalocean.com'
    port = 25060
    database = 'MandatoryMetadata'
    schema = 'testing'

    conn = psycopg2.connect(host=host, database=database,
                            user=username, password=password, port=port)
    cur = conn.cursor()
    cur.execute("SET search_path TO " + schema)

    return conn, cur

In [30]:
conn,cur = get_db_cursor()

In [4]:
def convertSpectra(df):
    print("Converting spectra")
    df_ = pd.DataFrame([i[[i for i in i.keys()][0]] for i in df['averaged_spectra'].values],columns = np.arange(522,3977,2))
    df_.index = df.index
    print("Spectra converted")
    return df_

In [5]:
def getValidAver(sample_codes, subset_count=None, filename="spectraldata"):
    conn, cur = get_db_cursor()
    spectra = pd.DataFrame(columns=['sample_code','averaged_spectra'])
    # count = len(sample_codes)
    if(len(sample_codes) < 5000):
        count  = len(sample_codes)
        step=count
    elif(len(sample_codes) < 70000):
        count = len(sample_codes)
        step=5000
    else:
        count = 100000
        step=5000
    start = 0

    for i in np.arange(start, count, step):
        
        print("Fetching spectra from {}".format(start))
        samples = [i for i in sample_codes][start:start+step]
        query = f"SELECT spectraldata.metadata_id, averaged_spectra, mandatorymetadata.sample_code  FROM spectraldata INNER JOIN mandatorymetadata ON mandatorymetadata.metadata_id = spectraldata.metadata_id WHERE is_finalized=True AND passed=True AND is_active=True AND averaged=True AND sample_code IN {str(samples).replace('[','(').replace(']',')')}"
        
        _ = pd.read_sql(query, con=conn)
        spectra = pd.concat([spectra, _], axis=0)
        start = start + step
        if (count-step) > 5000:
            step=5000
        else:
            step = count-step

    conn.close()
    spectra = spectra[['sample_code', 'averaged_spectra']]
    spectra = spectra.set_index('sample_code')
    spectra = convertSpectra(spectra)


    spectra.to_csv(f"input/{filename}.csv")

    return spectra

In [22]:
def getSampleCodes():
    df = pd.read_sql("""
        select m.sample_code, f.validated_subtype from fertilizercomments f 
        inner join spectraldata s 
        on s.spectral_data_id = f.spectral_data_id 
        inner join mandatorymetadata m 
        on m.metadata_id = s.metadata_id 
        where f.has_validated_subtype = true 
    """,con=conn)
    return df

In [63]:
def getCategory(value, lower, median, upper):
        if (value <= lower):
            return 1
        elif (value <= median):
            return 2
        elif (value <= upper):
            return 3
        else:
            return 4

In [77]:
def hyperparamTuning(X_valid, X_test, y_valid, y_test):
    search_space = {
        "n_estimators": tune.choice([50, 100, 150, 200, 250]),
        "max_depth": tune.choice([None, 10, 20, 30, 40 ,50, 60]),
        "min_samples_split": tune.choice([2, 5, 10, 12,15,20,32,36,40]),
        "criterion": tune.choice(["gini", "entropy", "log_loss"]),
        "min_samples_leaf": tune.randint(1, 15),
        "max_features": tune.choice(["sqrt", "log2", None]),
        "X_valid": X_valid,
        "X_test": X_test,
        "y_valid": y_valid,
        "y_test": y_test,
    }
    analysis = tune.run(
        trainRF,
        config=search_space,
        metric="accuracy",
        mode="max",
        num_samples=200,
    )

    

    best_config = analysis.get_best_config(metric="accuracy", mode="max")

    del best_config['X_valid']
    del best_config['y_valid']
    del best_config['X_test']
    del best_config['y_test']

    return best_config

In [95]:
def trainRF(config):
    valid_features = config['X_valid']
    valid_target = config['y_valid']
    test_features = config['X_test']
    test_target = config['y_test']
    del config['X_valid']
    del config['y_valid']
    del config['X_test']
    del config['y_test']
    model = RandomForestClassifier(**config)
    model.fit(valid_features, valid_target)
    score = model.score(test_features, test_target)
    return {'accuracy': score}

In [96]:
def trainRFClassifier(best_config, X_train, y_train):
    rf = RandomForestClassifier(**best_config)
    rf.fit(X_train, y_train)

    pickle.dump(rf, open("output/RandomForestClassifier.pkl","wb"))

    return rf

In [16]:
 query = f"SELECT spectraldata.metadata_id, averaged_spectra, mandatorymetadata.sample_code  FROM spectraldata INNER JOIN mandatorymetadata ON mandatorymetadata.metadata_id = spectraldata.metadata_id WHERE is_finalized=True AND passed=True AND is_active=True AND averaged=True LIMIT 1000"
_ = pd.read_sql(query, con=conn)
_ = _.set_index("sample_code")
df = convertSpectra(_)
df.to_csv("input/spectraldata.csv")
split("D://Cropnuts/DSML141/input/spectraldata.csv",
          "D://Cropnuts/DSML141/input/rds/", "D://Cropnuts/DSML141/input/splits", "data")
sc = StandardScaler()
df_scaled = sc.fit_transform(df)
df_scaled = pd.DataFrame(df_scaled,index=df.index,columns=df.columns)
df_quartiles = pd.DataFrame(index=df_scaled.index)
quartiles_dict = {}
for col in df_scaled.columns:
    lower = df_scaled[col].quantile(0.25)
    median = df_scaled[col].median()
    upper = df_scaled[col].quantile(0.75)
    quartiles_dict[col] = {
            "lower":lower,
            "median":median,
            "upper":upper
    }
    df_quartiles[col] =  df_scaled[col].apply(getCategory, args=( lower, median, upper))

train = pd.read_csv("input/splits/data_train_sample_codes.csv",index_col=1).index
test = pd.read_csv("input/splits/data_test_sample_codes.csv",index_col=1).index
valid = pd.read_csv("input/splits/data_valid_sample_codes.csv",index_col=1).index

X_train = df_quartiles.loc[df_quartiles.index.isin(train)]
X_test = df_quartiles.loc[df_quartiles.index.isin(test)]
X_valid = df_quartiles.loc[df_quartiles.index.isin(valid)]

y_train = np.random.randint(0,10, size=(len(X_train)))
y_test = np.random.randint(0,10, size=(len(X_test)))
y_valid = np.random.randint(0,10, size=(len(X_valid)))

C:\Python312\Lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [ ]:
best_config = hyperparamTuning(X_valid, X_test, y_valid, y_test)

2024-07-05 04:26:51,285	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


2024-07-05 04:26:51,773	WARNING trial.py:647 -- The path to the trial log directory is too long (max length: 260. Consider using `trial_dirname_creator` to shorten the path. Path: C:\Users\tommy\AppData\Local\Temp\ray\session_2024-07-05_04-24-11_703299_7056\artifacts\2024-07-05_04-26-51\trainRF_2024-07-05_04-26-51\driver_artifacts\trainRF_a7ce0_00000_0_criterion=log_loss,max_depth=40,max_features=log2,min_samples_leaf=1,min_samples_split=40,n_estimators=150_2024-07-05_04-26-51
2024-07-05 04:26:51,778	WARNING trial.py:647 -- The path to the trial log directory is too long (max length: 260. Consider using `trial_dirname_creator` to shorten the path. Path: C:\Users\tommy\AppData\Local\Temp\ray\session_2024-07-05_04-24-11_703299_7056\artifacts\2024-07-05_04-26-51\trainRF_2024-07-05_04-26-51\driver_artifacts\trainRF_a7ce0_00000_0_criterion=log_loss,max_depth=40,max_features=log2,min_samples_leaf=1,min_samples_split=40,n_estimators=150_2024-07-05_04-26-51
2024-07-05 04:26:51,783	WARNING tria

In [31]:
df_validated_subtypes = getSampleCodes()

In [34]:
train=[]
test=[]
valid=[]
for i in df_validated_subtypes.validated_subtype.unique():
    subtype_sample_codes = df_validated_subtypes.loc[df_validated_subtypes['validated_subtype'] == i]
    subtype_spc = getValidAver(subtype_sample_codes['sample_code'],filename=i)
    split(f"D://Cropnuts/DSML155/input/{i}.csv",
          "D://Cropnuts/DSML155/input/rds/", "D://Cropnuts/DSML155/input/splits", i)
    train.extend([i for i in pd.read(f"{i}_train_sample_codes.csv",index_col=1).index])
    test.extend([i for i in pd.read(f"{i}_test_sample_codes.csv",index_col=1).index])
    valid.extend([i for i in pd.read(f"{i}_valid_sample_codes.csv",index_col=1).index])

pandas.core.indexes.base.Index

In [46]:
le = LabelEncoder()
y = le.fit_transform(df_validated_subtypes.validated_subtype)
pickle.dump(le, open("output/target_le.pkl","wb"))

In [48]:
spc = getValidAver(df_validated_subtypes['sample_code'])

In [56]:
sc = StandardScaler()
spc_scaled = sc.fit_transform(spc)
spc_scaled = pd.DataFrame(spc_scaled, index=spc.index, columns=spc.columns)
pickle.dump(sc, open("output/feature_scaler.pkl","wb"))

In [ ]:
spc_quartiles = pd.DataFrame(index=spc_scaled.index)
quartiles_dict = {}
for col in spc_scaled.columns:
    lower = spc_scaled[col].quantile(0.25)
    median = spc_scaled[col].median()
    upper = spc_scaled[col].quantile(0.75)
    quartiles_dict[col] = {
            "lower":lower,
            "median":median,
            "upper":upper
    }
    spc_quartiles[col] =  spc_scaled[col].apply(getCategory, args=( lower, median, upper))
pickle.dump(quartiles_dict, open("output/absorbance_quartiles.dict","wb"))

In [ ]:
X_train = spc_quartiles.loc[spc_quartiles.index.isin(train)]
X_test = spc_quartiles.loc[spc_quartiles.index.isin(test)]
X_valid = spc_quartiles.loc[spc_quartiles.index.isin(valid)]

y_train = y.loc[spc_quartiles.index.isin(train)]
y_test = y.loc[spc_quartiles.index.isin(test)]
y_valid = y.loc[spc_quartiles.index.isin(valid)]

In [ ]:
best_config = hyperparamTuning(X_valid, X_test, y_valid, y_test)

In [ ]:
clf = trainRFClassifier(best_config, X_train, y_train)